<a href="https://colab.research.google.com/github/zhannatoleubek-png/special-okx-chainsaw/blob/main/%D0%A0%D0%B0%D0%B7%D0%B2%D0%BE%D1%80%D0%BE%D1%82%20%D1%83%D0%BB%D1%83%D1%87%D1%88%D0%B5%D0%BD%D0%BD%D1%8B%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================
# Надёжный OKX-анализатор + Telegram (synchronous)
# ===============================
# Требуемые библиотеки: requests, pandas, numpy, ta
!pip install -q requests pandas numpy ta

import time
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timezone
from ta.trend import EMAIndicator, MACD
from ta.momentum import RSIIndicator

# -------------------------------
# Настройки — ВСТАВЬ СВОИ ДАННЫЕ
# -------------------------------
TOKEN = "8294892098:AAFX0Zzq9yN1on6UlID8f7vzif4dWR_7uWs"   # <-- вставь токен
CHAT_ID = "343376986"            # <-- вставь chat_id (строка или число)
ANALYSIS_INTERVAL = 10 * 60        # интервал полного прохода (сек) — 10 минут
TOP_N = 8                          # сколько топ-активов анализировать
INTERVALS = ["3m", "15m", "1h", "4h", "12h", "1d"]

OKX_TICKERS_URL = "https://www.okx.com/api/v5/market/tickers?instType=SWAP"
OKX_CANDLES_URL = "https://www.okx.com/api/v5/market/candles"  # params: instId, bar, limit

# Максимально-допустимый возраст последней свечи (коэффициент от размера бара)
FRESHNESS_FACTOR = 2  # если возраст > interval_seconds * FRESHNESS_FACTOR => считаем устаревшим

# соответствие бара к секундам
INTERVAL_TO_SECONDS = {
    "3m": 3 * 60,
    "15m": 15 * 60,
    "1h": 60 * 60,
    "4h": 4 * 60 * 60,
    "12h": 12 * 60 * 60,
    "1d": 24 * 60 * 60,
}

# -------------------------------
# Утилиты Telegram (через HTTP)
# -------------------------------
TELEGRAM_API = f"https://api.telegram.org/bot{TOKEN}"

def tg_send(text):
    """Отправить сообщение в Telegram через Bot API (synchronous)."""
    try:
        r = requests.post(TELEGRAM_API + "/sendMessage", data={"chat_id": CHAT_ID, "text": text}, timeout=10)
        j = r.json()
        if not j.get("ok", False):
            print("❌ Telegram API вернул ошибку:", j)
            return False
        return True
    except Exception as e:
        print("❌ Ошибка отправки в Telegram:", e)
        return False

def tg_check():
    """Проверить токен и чат — возвращает (bot_info, chat_ok_bool)."""
    try:
        me = requests.get(TELEGRAM_API + "/getMe", timeout=10).json()
        if not me.get("ok"):
            print("❌ Некорректный токен (getMe):", me)
            return None, False
        bot_username = me["result"].get("username")
        # проверим доступ к чату
        chat_resp = requests.get(TELEGRAM_API + "/getChat", params={"chat_id": CHAT_ID}, timeout=10).json()
        chat_ok = chat_resp.get("ok", False)
        if not chat_ok:
            print("⚠️ Бот не имеет доступа к указанному chat_id (getChat вернул):", chat_resp)
        return bot_username, chat_ok
    except Exception as e:
        print("❌ Ошибка проверки Telegram:", e)
        return None, False

# -------------------------------
# OKX функции
# -------------------------------
def get_top_active_symbols(limit=TOP_N):
    """Возвращает список instId самых активных свопов (по vol24h)."""
    try:
        resp = requests.get(OKX_TICKERS_URL, timeout=10).json()
        data = resp.get("data") or []
        if not data:
            print("⚠️ OKX: пустой ответ на тикеры.")
            return []
        df = pd.DataFrame(data)
        # Некоторые поля могут быть строками; преобразуем аккуратно
        if "vol24h" in df.columns:
            df["vol24h"] = pd.to_numeric(df["vol24h"], errors="coerce").fillna(0.0)
        else:
            df["vol24h"] = 0.0
        # Используем instId
        df = df.sort_values("vol24h", ascending=False)
        return df["instId"].head(limit).tolist()
    except Exception as e:
        print("❌ Ошибка при получении топ-активов OKX:", e)
        return []

def get_klines_okx(symbol, interval, limit=200):
    """
    Возвращает DataFrame свечей в хронологическом порядке (старые->новые).
    OKX возвращает список списков: [ts, o, h, l, c, vol, ...]
    """
    try:
        params = {"instId": symbol, "bar": interval, "limit": str(limit)}
        r = requests.get(OKX_CANDLES_URL, params=params, timeout=10).json()
        data = r.get("data") or []
        if not data:
            return None
        # Если OKX вернул массив списков — создаём DataFrame с колонками
        df = pd.DataFrame(data)
        if df.shape[1] >= 5:
            df = df.iloc[:, :9]
            df.columns = ["ts", "o", "h", "l", "c", "vol", "volCcy", "volCcyQuote", "confirm"][:df.shape[1]]
        # Приводим числовые колонки
        for col in ["o", "h", "l", "c", "vol"]:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors="coerce")
        # ts — обычно миллисекунды. Приведём в int:
        df["ts"] = df["ts"].astype(float).astype("Int64")
        # хронологический порядок: старые -> новые
        df = df.iloc[::-1].reset_index(drop=True)
        return df
    except Exception as e:
        print(f"❌ Ошибка загрузки свечей OKX для {symbol} {interval}:", e)
        return None

# -------------------------------
# Анализ (с проверкой "актуальности" свечи)
# -------------------------------
def is_fresh(df, interval):
    """Проверяем, что последняя свеча достаточно свежая."""
    if df is None or df.empty:
        return False, "нет данных"
    last_ts_ms = int(df["ts"].iloc[-1])
    age_seconds = (time.time() * 1000 - last_ts_ms) / 1000.0
    allowed = INTERVAL_TO_SECONDS.get(interval, 60) * FRESHNESS_FACTOR
    if age_seconds > allowed:
        return False, f"последняя свеча устарела ({int(age_seconds)}s > {int(allowed)}s)"
    return True, f"свежая ({int(age_seconds)}s)"

def analyze_trend_df(df):
    """Анализ тренда по df (ожидается колонка 'c' — цены закрытия)."""
    try:
        if df is None or len(df) < 50:
            return None
        close = df["c"].astype(float).reset_index(drop=True)
        ema_short = EMAIndicator(close, window=9).ema_indicator()
        ema_long = EMAIndicator(close, window=26).ema_indicator()
        macd_obj = MACD(close)
        macd = macd_obj.macd()
        macd_signal = macd_obj.macd_signal()
        rsi = RSIIndicator(close).rsi()

        last = -1
        prev = -2

        # простая логика разворота
        if ema_short.iloc[last] > ema_long.iloc[last] and ema_short.iloc[prev] <= ema_long.iloc[prev] and rsi.iloc[last] < 70:
            return "📈 Сильный разворот вверх"
        if ema_short.iloc[last] < ema_long.iloc[last] and ema_short.iloc[prev] >= ema_long.iloc[prev] and rsi.iloc[last] > 30:
            return "📉 Сильный разворот вниз"
        return None
    except Exception as e:
        print("❌ Ошибка в analyze_trend_df:", e)
        return None

# -------------------------------
# Основной проход анализа
# -------------------------------
def run_once():
    now_str = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S UTC")
    print(f"\n=== Запуск анализа: {now_str} ===")
    active = get_top_active_symbols(limit=TOP_N)
    if not active:
        print("⚠️ Нет активов для анализа.")
        return

    print("📊 Топ активов:", active)
    for symbol in active:
        try:
            signals_for_symbol = {}
            for tf in INTERVALS:
                df = get_klines_okx(symbol, tf, limit=200)
                fresh, reason = is_fresh(df, tf)
                if not fresh:
                    print(f"  {symbol} {tf}: пропускаем — {reason}")
                    continue
                # анализ
                trend = analyze_trend_df(df)
                if trend:
                    signals_for_symbol[tf] = trend
                    print(f"  {symbol} {tf}: {trend}")
                else:
                    print(f"  {symbol} {tf}: нет сигнала (стандартный анализ)")
            # если есть сигналы — отправляем сообщение
            if signals_for_symbol:
                txt = f"🔔 Сигнал по {symbol}\n"
                for tf, t in signals_for_symbol.items():
                    txt += f"⏱ {tf}: {t}\n"
                txt += f"\n🕒 {now_str}\n(аналитика по актуальным данным OKX)"
                ok = tg_send(txt)
                print("   -> отправлено в Telegram:", ok)
        except Exception as e:
            print(f"❌ Ошибка при обработке {symbol}:", e)
            tg_send(f"⚠️ Ошибка обработки {symbol}: {e}")

# -------------------------------
# Запуск (главная логика)
# -------------------------------
if __name__ == "__main__":
    print("Проверяем Telegram токен и доступ к чату...")
    bot_name, chat_ok = tg_check()
    if bot_name:
        print("Bot username:", bot_name)
    if not bot_name or not chat_ok:
        print("⚠️ Проверь токен/CHAT_ID или начни диалог с ботом в Telegram. Скрипт продолжит, но сообщения могут не доходить.")
    else:
        print("✅ Telegram проверен — бот и чат доступны.")

    # тестовое сообщение
    print("Отправляем тестовое сообщение...")
    if tg_send("🚀 Бот запущен — тестовое сообщение. Анализ начнётся вскоре."):
        print("✅ Тестовое сообщение отправлено (проверь Telegram).")
    else:
        print("❌ Тестовое сообщение не отправлено — проверь токен/CHAT_ID и начни чат с ботом.")

    # Главный цикл — синхронный, с печатью прогресса
    try:
        while True:
            run_once()
            # Делаем видимый счётчик ожидания: каждую минуту печатаем оставшееся
            total = ANALYSIS_INTERVAL
            if total <= 0:
                break
            minutes = total // 60
            for i in range(minutes):
                print(f"⏳ Ожидание следующего прохода: {i+1}/{minutes} минут ({(minutes-i-1)} минут осталось)")
                time.sleep(60)
            # если ANALYSIS_INTERVAL не кратен минутам — досыпем остаток
            remainder = total % 60
            if remainder:
                time.sleep(remainder)
    except KeyboardInterrupt:
        print("Остановлено вручную.")
    except Exception as e:
        print("Фатальная ошибка в основном цикле:", e)
        tg_send(f"🚨 Фатальная ошибка скрипта: {e}")

Проверяем Telegram токен и доступ к чату...
Bot username: stellium_astro_bot
✅ Telegram проверен — бот и чат доступны.
Отправляем тестовое сообщение...
❌ Telegram API вернул ошибку: {'ok': False, 'error_code': 403, 'description': 'Forbidden: bot was blocked by the user'}
❌ Тестовое сообщение не отправлено — проверь токен/CHAT_ID и начни чат с ботом.

=== Запуск анализа: 2025-10-08 18:54:26 UTC ===
📊 Топ активов: ['ACT-USDT-SWAP', 'W-USDT-SWAP', 'ICP-USDT-SWAP', 'FIL-USDT-SWAP', 'STRK-USDT-SWAP', 'BIO-USDT-SWAP', 'ASTER-USDT-SWAP', 'CELO-USDT-SWAP']
  ACT-USDT-SWAP 3m: 📈 Сильный разворот вверх
  ACT-USDT-SWAP 15m: нет сигнала (стандартный анализ)
  ACT-USDT-SWAP 1h: пропускаем — нет данных
  ACT-USDT-SWAP 4h: пропускаем — нет данных
  ACT-USDT-SWAP 12h: пропускаем — нет данных
  ACT-USDT-SWAP 1d: пропускаем — нет данных
❌ Telegram API вернул ошибку: {'ok': False, 'error_code': 403, 'description': 'Forbidden: bot was blocked by the user'}
   -> отправлено в Telegram: False
  W-USDT-SWAP 